In [6]:
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras import backend as K
import numpy as np
import pandas as pd
import os

In [7]:
def feature_extract(data_path, model_name, save_path, model_path, step_num, fold_no, k, class_type):
    print(f"Processing: {model_name}")
    model = load_model(os.path.join(model_path, model_name))
    acc=model_name.split("acc")[1].split(".")[0]

    # Flatten 또는 Dense Layer 찾기
    dense_layer_names = ['dense_1', 'dense_4', 'dense_7']
    flatten = None
    for layer_name in dense_layer_names:
        try:
            flatten = model.get_layer(layer_name)
            break
        except:
            continue

    if flatten is None:
        raise ValueError("No suitable Dense layer found in the model.")

    test_model = Model(inputs=model.input, outputs=flatten.output)

    # 클래스 디렉토리 설정
    img_dirs = {cls: os.path.join(data_path, cls) for cls in class_type}
    img_features_all = []

    for cls_label, cls_name in enumerate(class_type):
        img_dir = img_dirs[cls_name]
        img_list = sorted(os.listdir(img_dir))

        for img_name in img_list:
            img_path = os.path.join(img_dir, img_name)
            img = image.load_img(img_path, target_size=(300, 300))
            img_tensor = image.img_to_array(img) / 255.0
            img_tensor = np.expand_dims(img_tensor, axis=0)

            # 특징 추출
            img_features1 = test_model.predict(img_tensor)[0]
            img_features2 = np.append(img_features1, [img_name, cls_name])
            img_features_all.append(img_features2)

    # 레이블 생성
    total_labels = np.array([cls_label for cls_label, cls_name in enumerate(class_type) for _ in range(len(os.listdir(img_dirs[cls_name])))])
    feature_set = [np.append(img_features_all[i], total_labels[i]) for i in range(len(total_labels))]

    # Feature column name
    feature_columns = [f'img_f{i}_{step_num}' for i in range(len(img_features_all[0]))] + ['AA']
    #create_directory(save_path)

    # 데이터프레임 저장
    df = pd.DataFrame(feature_set, columns=feature_columns)
    csv_filename = f"img_features_{acc}_q{step_num}_{k}{fold_no}_{class_type[0]}vs{class_type[1]}.csv"
    df.to_csv(os.path.join(save_path, csv_filename), index=False)
    print(f"Saved: {csv_filename}")

In [ ]:
# 설정
# 추출할 데이터셋 설정
dataset ="dataset20250127"
class_types = ["normal_abnormal", "MCI_AD",  "SCI_AD", "SCI_MCI"]

for class_type in class_types:
    class_dirs = class_type.split('_')  # 클래스 이름 추출
    for step_no in range(1, 12):
        for k in ('train', 'test'):
            for fold_num in range(1, 6):
                save_path = f'./flatten/{class_type}/'
                os.makedirs(save_path,exist_ok=True)
                model_path = f'./models/{class_type}/'
                data_path = f'/104data/dataset/음성/voice_to_image/{dataset}/{class_type}/{step_no}/iterations/iteration{fold_num}/{k}/'

                model_list = os.listdir(model_path)
                model_list = sorted(filter(lambda x: f"q{step_no}_fold{fold_num}" in x, model_list))

                for model_name in model_list:
                    if model_name.endswith('.h5'):
                        feature_extract(data_path, model_name, save_path, model_path, step_no, fold_num, k, class_dirs)
                        print(f"{step_no}, Fold {fold_num}, {k} is done")
                        K.clear_session()


Processing: model_normal_abnormal_q1_fold1_acc50.0.h5
Saved: img_features_50_q1_train1_normalvsabnormal.csv
1, Fold 1, train is done
Processing: model_normal_abnormal_q1_fold2_acc50.0.h5
Saved: img_features_50_q1_train2_normalvsabnormal.csv
1, Fold 2, train is done
Processing: model_normal_abnormal_q1_fold3_acc50.0.h5
Saved: img_features_50_q1_train3_normalvsabnormal.csv
1, Fold 3, train is done
Processing: model_normal_abnormal_q1_fold4_acc50.0.h5
Saved: img_features_50_q1_train4_normalvsabnormal.csv
1, Fold 4, train is done
Processing: model_normal_abnormal_q1_fold5_acc50.0.h5
Saved: img_features_50_q1_train5_normalvsabnormal.csv
1, Fold 5, train is done
Processing: model_normal_abnormal_q1_fold1_acc50.0.h5
Saved: img_features_50_q1_test1_normalvsabnormal.csv
1, Fold 1, test is done
Processing: model_normal_abnormal_q1_fold2_acc50.0.h5
Saved: img_features_50_q1_test2_normalvsabnormal.csv
1, Fold 2, test is done
Processing: model_normal_abnormal_q1_fold3_acc50.0.h5
Saved: img_features

FileNotFoundError: [Errno 2] No such file or directory: './models/MCI_AD/'

In [ ]:
# 베스트로만 진행할경우 이곳에서 진행.
# 추출할 데이터셋 설정
dataset ="dataset20250127"
class_types = ["normal_abnormal", "MCI_AD",  "SCI_AD", "SCI_MCI"]

for class_type in class_types:
    class_dirs = class_type.split('_')  # 클래스 이름 추출
    for step_no in range(1, 12):
        for k in ('train', 'test'):
            for fold_num in range(1, 6):
                save_path = f'./flatten/{class_type}/'
                os.makedirs(save_path,exist_ok=True)
                model_path = f'./selected_bestmodel/{class_type}/'
                data_path = f'/104data/dataset/음성/voice_to_image/{dataset}/{class_type}/{step_no}/iterations/iteration{fold_num}/{k}/'

                model_list = os.listdir(model_path)
                model_list = sorted(filter(lambda x: f"q{step_no}" in x, model_list))

                for model_name in model_list:
                    if model_name.endswith('.h5'):
                        feature_extract(data_path, model_name, save_path, model_path, step_no, fold_num, k, class_dirs)
                        print(f"{step_no}, Fold {fold_num}, {k} is done")
                        K.clear_session()
